# Datenanalyse der Brettspielwelt-DB

## Datenbestand

### Datenbeschaffung

Es stehen ca. 7 GB Logdateien vom Spiele-Portal "Brettspielwelt" zur Verfügung (seit 2007-01, alle Logdateien bis einschließlich 2025-07).
Jede Logdatei hat die Daten einer Partie.

Nach dem Download wurde jede Logdatei geparst (in ein Objekt eingelesen), validiert und in eine SQLite-DB gepumpt.
Dieser Vorgang dauerte 16 Stunden und 37 Minuten (ca. 25ms / Partie).

Dabei wurde gezählt:
*   2.411.514 Logdateien insgesamt
*   13.964 leere Logdateien (ohne verwertbare Daten)

Die SQLite-Datenbankdatei ist 14.842.700 KB = 14,155 GB groß.

### Anzahl Partien

*   2.397.550 Partien insgesamt
*   2.141.248 fehlerfreie Partien



In [21]:
%%sql
select count(*) as Partien from games

,Partien
0,2397550


In [35]:
%%sql
select g.error_code as Code, e.name as Fehler, count(*) as Anzahl, e.description as Beschreibung
from games as g
left join errors as e on g.error_code = e.code
group by g.error_code
order by g.error_code

,Code,Fehler,Anzahl,Beschreibung
0,0,NO_ERROR,2297206,Kein Fehler
1,12,DUPLICATE_CARD,6,Karten mehrmals vorhanden
2,21,WISH_NOT_FOLLOWED,79,Wunsch nicht beachtet
3,23,SMALLER_OF_AMBIGUOUS_RANK,10807,Es wurde der kleinere Rang bei einem mehrdeuti...
4,24,PLAYER_NOT_ON_TURN,2,Der Spieler ist nicht am Zug
5,26,HISTORY_TOO_LONG,55,"Karten ausgespielt, obwohl die Runde vorbei is..."
6,31,DRAGON_GIVEN_TO_OWN_TEAM,276,Drache an eigenes Team verschenkt
7,50,ANNOUNCEMENT_NOT_POSSIBLE,272,Tichu-Ansage an der geloggten Position nicht m...
8,60,SCORE_MISMATCH,1401,Geloggtes Rundenergebnis stimmt nicht mit dem ...
9,61,SCORE_NOT_POSSIBLE,8862,Rechenfehler! Geloggtes Rundenergebnis ist nic...


In [2]:
%%sql
select count(*) from games where error_code = 0 and player_changed = 0

,count(*)
0,2141248


**SCORE_NOT_POSSIBLE:** Der Rechenfehler wurde 2010 größtenteils behoben:

In [45]:
%%sql
SELECT
    g.log_year AS Jahr,
    COUNT(CASE WHEN error_code = 61 THEN 1 END) AS Rechenfehler,
    COUNT(*) AS Total_Games,
    ROUND((COUNT(CASE WHEN error_code = 61 THEN 1 END) * 100.0) / COUNT(*), 2) AS [Fehlerrate in %]
FROM
    games AS g
GROUP BY
    log_year
ORDER BY
    log_year;

,Jahr,Rechenfehler,Total_Games,Fehlerrate in %
0,2007,927,94969,0.98
1,2008,1944,147438,1.32
2,2009,3111,180640,1.72
3,2010,2794,219729,1.27
4,2011,27,329552,0.01
5,2012,23,397078,0.01
6,2013,8,288380,0.00
7,2014,5,195838,0.00
8,2015,7,136494,0.01
9,2016,5,100815,0.00


## Runden

*   22.458.253 Runden insgesamt.
*   20.186.515 Runden unter den fehlerfreien Partien (durchschnittlich 9,4 Runden / Partie).

In [48]:
%%sql
select count(*) as Runden from rounds

,Runden
0,22458253


In [6]:
%%sql
select count(*) as Runden
from rounds as r
inner join games as g on g.id = r.game_id
where g.error_code = 0 and g.player_changed = 0

,Runden
0,20186515


In [3]:
20186515 / 2141248

9.427453055414412

### Spieler

*   128.462 Spieler insgesamt
*   110.932 Spieler, die eine fehlerfreie Partie durchgespielt haben

In [9]:
%%sql
select count(*) as Spieler from players

,Spieler
0,128462


In [24]:
%%sql
select count(*)
from (
select player_id_0 from games where error_code = 0 and player_changed = 0
union
select player_id_1 from games where error_code = 0 and player_changed = 0
union
select player_id_2 from games where error_code = 0 and player_changed = 0
union
select player_id_3 from games where error_code = 0 and player_changed = 0
) as sub


,count(*)
0,110932
